In [446]:
from urllib.request import urlopen
import urllib.request
import pandas as pd
import numpy as np
import os
import re

In [416]:
i=2005
dfGOES = pd.DataFrame({'Class': [np.nan]})

In [500]:
i=2003
while i<=2003:
    target_url = 'https://www.ngdc.noaa.gov/stp/space-weather/solar-data/solar-features/solar-flares/x-rays/goes/xrs/goes-xrs-report_'+str(i)+'.txt'
    df = pd.read_fwf(target_url,header=None)
    currentCols = ['Date','NA1','NA2','NA3','NA4','Class','Magnitude','GOES']
    dropCols = ['NA1','NA2','NA3','NA4','GOES']
    if df.shape[1] == 9:
        currentCols.extend('NA5')
        dropCols.extend('NA5')
    elif df.shape[1] == 10:
        currentCols.extend(['NA5','NA6'])
        dropCols.extend(['NA5','NA6'])
    elif df.shape[1] == 11:
        currentCols.extend(['NA5','NA6','NA7'])
        dropCols.extend(['NA5','NA6','NA7'])
    elif df.shape[1] == 12:
        currentCols.extend(['NA5','NA6','NA7','NA8'])
        dropCols.extend(['NA5','NA6','NA7','NA8'])
    elif df.shape[1] == 13:
        currentCols.extend(['NA5','NA6','NA7','NA8','NA9'])
        dropCols.extend(['NA5','NA6','NA7','NA8','NA9'])
    df.columns = currentCols
    df = df.drop(dropCols, axis=1)
    df = df.drop(df.shape[0]-1, axis=0)
    #df['Date'] = pd.to_numeric(df.ID, errors='coerce')
    df[['Date']] = df[['Date']].apply(pd.to_numeric)
    df['Date'] = df['Date']-31777000000+20000000
    # In the new dataframe a hundreds digit of 1 corresponds the B class Events
    #                      a hundreds digit of 2 corresponds to C class Events
    #                      hundred digit of 3 ==> B
    #                      hundreds digit of 4 ==> X
    #df.loc[(df['Class']=='B'), 'Magnitude'] = df['Magnitude']+100
    #df.loc[(df['Class']=='C'), 'Magnitude'] = df['Magnitude']+200
    #df.loc[(df['Class']=='M'), 'Magnitude'] = df['Magnitude']+300
    #df.loc[(df['Class']=='X'), 'Magnitude'] = df['Magnitude']+400
    df = df.drop(['Magnitude'], axis=1)
    #print(df)
    dfGrouped=df.groupby('Date')
    dfGrouped = dfGrouped.max()
    #print(dfGrouped)
    dfGOES = dfGOES.append(dfGrouped)
    #print(dfGOES)
    i=i+1

In [501]:
dfGOES.dropna(axis=0,how = 'any')
#dfGOES['Class'][20050101.0]

,Class
20050101.0,X
20050102.0,C
20050103.0,C
20050104.0,C
20050105.0,B
20050106.0,B
20050107.0,B
20050108.0,C
20050109.0,M
20050110.0,C


In [502]:
year = 2003
month = 1
while year <=2003:
    month = 1
    while month <=12:
        isWack = False
        target_url2 = 'http://soi.stanford.edu/cgi-bin/mdi/gif_dir.pl?Directory=/synoptic/gifs/magnetograms/'+str(year)+'/'+ str(month).zfill(2)
        df2 = pd.read_fwf(target_url2,header=0, names = ['URL'])
        #print(df2['URL'][len(df2)-3])
        #df2.drop(df.index[len(df2['URL'])-1])
        #df2.drop(df.index[len(df2['URL'])-2])
        #df2.drop(df.index[len(df2['URL'])-3], inplace=False)
        df2['URL'][0] = df2['URL'][0][16:120]
        for index, row in df2.iterrows():
            try:
                df2['URL'][index+1]=df2['URL'][index+1][12:116]
            except KeyError:
                pass
            try:
                if df2['URL'][index][103] == 'g':
                    isWack = True
                    df2['URL'][index] = df2['URL'][index][:96]
                    #df2['URL'][index]=np.nan
            except IndexError:
                pass
        #df2.dropna(axis=0,how = 'any')
            
        #print(df2['URL'][122])
        for index, row in df2.iterrows():
            line = df2['URL'][index][76:86]
            #print(index)
            try:
                fullfilename = os.path.join('FINALDATA/'+ dfGOES['Class'][int(line.replace('.',''))], line.replace('.','') +'.jpeg')
                try:
                    if df2['URL'][index][:86]!= df2['URL'][index-1][:86]:
                        print(df2['URL'][index])
                        try:
                            urllib.request.urlretrieve(df2['URL'][index], fullfilename)
                        except urllib.request.HTTPError as err:
                            if err.code == 404:
                                pass
                except KeyError:
                    print(df2['URL'][index])
                    urllib.request.urlretrieve(df2['URL'][index], fullfilename)
            except (KeyError, ValueError, FileNotFoundError) as e:
                #print(index)
                pass
        month = month + 1
    year = year + 1

http://soi.stanford.edu/production/gif_images/magnetograms/2003/01//MDI_mag_2003.01.01_00:03.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/01//MDI_mag_2003.01.02_00:00.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/01//MDI_mag_2003.01.03_00:03.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/01//MDI_mag_2003.01.04_00:00.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/01//MDI_mag_2003.01.05_00:03.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/01//MDI_mag_2003.01.06_00:00.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/01//MDI_mag_2003.01.07_00:00.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/01//MDI_mag_2003.01.08_00:03.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/01//MDI_mag_2003.01.09_00:00.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/01//MDI_mag_2003.01.10_00:00.gif
http://soi.stanford.edu/produc

http://soi.stanford.edu/production/gif_images/magnetograms/2003/03//MDI_mag_2003.03.28_00:00.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/03//MDI_mag_2003.03.29_00:00.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/03//MDI_mag_2003.03.30_00:00.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/03//MDI_mag_2003.03.31_00:00.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/04//MDI_mag_2003.04.01_00:00.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/04//MDI_mag_2003.04.02_01:39.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/04//MDI_mag_2003.04.03_01:39.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/04//MDI_mag_2003.04.04_01:35.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/04//MDI_mag_2003.04.05_00:03.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/04//MDI_mag_2003.04.06_01:39.gif
http://soi.stanford.edu/produc

http://soi.stanford.edu/production/gif_images/magnetograms/2003/06//MDI_mag_2003.06.23_00:00.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/06//MDI_mag_2003.06.24_01:35.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/06//MDI_mag_2003.06.25_01:39.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/06//MDI_mag_2003.06.26_00:03.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/06//MDI_mag_2003.06.27_01:28.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/06//MDI_mag_2003.06.28_11:15.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/06//MDI_mag_2003.06.29_06:27.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/06//MDI_mag_2003.06.30_04:51.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/07//MDI_mag_2003.07.03_06:27.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/07//MDI_mag_2003.07.10_19:15.rotated.gif
http://soi.stanford.ed

http://soi.stanford.edu/production/gif_images/magnetograms/2003/09//MDI_mag_2003.09.21_00:00.rotated.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/09//MDI_mag_2003.09.22_00:00.rotated.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/09//MDI_mag_2003.09.23_01:35.rotated.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/09//MDI_mag_2003.09.24_00:03.rotated.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/09//MDI_mag_2003.09.25_00:03.rotated.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/09//MDI_mag_2003.09.26_00:03.rotated.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/09//MDI_mag_2003.09.27_00:03.rotated.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/09//MDI_mag_2003.09.28_00:00.rotated.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/09//MDI_mag_2003.09.29_00:51.rotated.gif
http://soi.stanford.edu/production/gif_images/magnetogr

http://soi.stanford.edu/production/gif_images/magnetograms/2003/12//MDI_mag_2003.12.17_00:03.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/12//MDI_mag_2003.12.18_00:03.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/12//MDI_mag_2003.12.19_01:35.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/12//MDI_mag_2003.12.20_01:35.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/12//MDI_mag_2003.12.21_01:35.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/12//MDI_mag_2003.12.22_01:35.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/12//MDI_mag_2003.12.23_00:03.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/12//MDI_mag_2003.12.24_00:03.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/12//MDI_mag_2003.12.25_01:39.gif
http://soi.stanford.edu/production/gif_images/magnetograms/2003/12//MDI_mag_2003.12.28_00:04.gif
http://soi.stanford.edu/produc